In [2]:
import pymongo

from string import punctuation

import charts

Server running in the folder d:\Backup\leiyue\Desktop\实战计划\Plan-for-combating\week3大作业提交\leiyue at 127.0.0.1:3262


In [3]:
client = pymongo.MongoClient('localhost', 27017)
scrapy = client['scrapy']
item_info = scrapy['item_infoY']

In [4]:
for i in item_info.find({'$and': [{'look': {'$ne' : '-'}}, {'look': {'$ne' : '${info.paramsMap.oldlevel}'}}]}).limit(300):
        print(i['price'])


900
1600
50
60
300
80
150
8
500
550
700
650
None
450
1250
200
1000
50
380
2400
1288
300
320
1000
1000
12
900
950
800
500
3500
700
1000
200
900
360
None
1000
199
300
368
30
400
300
None
500
300
1600
300
40
300
88
600
300
150
300
300
500
None
700
300
50
170
400
300
700
300
750
1080
10
300
450
380
260
None
150
300
1000
228
300
200
100
800
200
400
6500
280
300
200
300
700
280
300
300
None
300
300
160
1600
280
120
380
120
280
300
280
300
220
85
None
700
280
300
280
280
260
30
20
280
300
300
290
2400
280
7000
280
160
1200
100
300
280
150
180
550
290
700
30
120
1200
300
300
800
750
180
350
160
300
350
300
70
280
260
400
750
280
1600
1
None
12000
270
300
800
300
3500
698
2600
2300
70
10
4360
110
680
1000
9500
4200
15000
8300
3910
200
150
200
4800
500
1200
600
220
3900
4800
4000
14800
700
1200
450
700
3800
4800
80
700
580
3800
100
1300
3800
80
20
8500
600
65
6300
300
1280
160
3800
9500
1100
70
36800
550
550
36800
380
380
26800
50
8000
8
600
26800
7000
20
2500
1200
26500
3000
12
300
8000
25
1500

In [5]:
for i in item_info.find():
    if type(i['price']) is str:
        exclude = set(punctuation)
        s = ''.join(ch for ch in i['price'] if ch not in exclude)
        frags= s.split('元')
        if not frags:
            price = None
        elif frags[0] == '面议':
            price = None
        elif frags[0] == '${info.paramsMap.minprice}':
            price = None
        else:
            price = int(frags[0])
    else:
        price = i['price']
    item_info.update_one(dict(_id=i['_id']),{'$set': dict(price=price)})

In [20]:
look_list = []
for i in item_info.find({'$and': [{'look': {'$ne' : '-'}}, {'look': {'$ne' : '${info.paramsMap.oldlevel}'}}]}):
    look_list.append(i['look'])

look_index = list(set(look_list))
print(look_index)

['报废机/尸体', '7成新及以下', '95成新', '99成新', '8成新', '全新', '9成新']


In [19]:
categories = ['全新', '99成新', '95成新', '9成新', '8成新', '7成新及以下', '报废机/尸体']
cate='北京二手手机'

index = dict()
for value, key in enumerate(categories):
    index[key] = value


pipeline = [
    {'$match': {'$and': [dict(cates=cate), dict(look={'$ne': '-'}), dict(look={'$ne': '${info.paramsMap.oldlevel}'})]}},
    {'$group': dict(_id='$look', counts={'$sum': 1}, avgPrice={'$avg': '$price'})},
    {'$sort': dict(_id=pymongo.DESCENDING)},
    {'$limit': 99}
]

look_prices = []
prices = []

for i in item_info.aggregate(pipeline):
    item=dict(category=i['_id'],data=i['avgPrice'],index=index[i['_id']])
    look_prices.append(item)

sorted_look_prices = sorted(look_prices, key = lambda item: (item['index']))
for i in sorted(look_prices, key = lambda item: (item['index'])):
    prices.append(i['data'])

print(categories)
print(prices)

options = dict(
    title=dict(text='成色价格统计'),
    chart=dict(zoomType='xy'),
    xAxis=dict(title=dict(text='成色'), categories=categories),
    yAxis=dict(title=dict(text='价格'))
)

series = [{
        'name': cate,
        'data': prices,
        'type': 'line' 
    }]

charts.plot(series, options=options, show='inline')

['全新', '99成新', '95成新', '9成新', '8成新', '7成新及以下', '报废机/尸体']
[1922.5413533834587, 1681.8417721518988, 2231.519047619048, 1073.1206896551723, 633.982905982906, 558.4615384615385, 495.0]


In [20]:
cate_list = []
for i in item_info.find():
    cate_list.append(i['cates'][2])
cate_index = list(set(cate_list))
print(cate_index)

['北京二手手机', '北京二手母婴/儿童用品', '北京二手设备', '北京二手图书/音像/软件', '北京二手平板电脑', '北京二手台式机/配件', '北京二手服装/鞋帽/箱包', '北京二手办公用品/设备', '北京二手家具', '北京其他二手物品', '北京二手家电', '北京二手笔记本', '北京二手数码产品', '北京二手美容/保健', '北京二手文体/户外/乐器']


In [23]:
def generator(types):
    for cate in cate_index:
        pipeline = [
            {'$match': {'$and': [dict(cates=cate), dict(look={'$ne': '-'}), dict(look={'$ne': '${info.paramsMap.oldlevel}'})]}},
            {'$group': dict(_id='$look', counts={'$sum': 1}, avgPrice={'$avg': '$price'})},
            {'$sort': dict(_id=pymongo.DESCENDING)},
            {'$limit': 99}
        ]
        look_prices = []
        prices = []

        for i in item_info.aggregate(pipeline):
            item=dict(category=i['_id'],data=i['avgPrice'],index=index[i['_id']])
            look_prices.append(item)

        sorted_look_prices = sorted(look_prices, key = lambda item: (item['index']))
        for i in sorted(look_prices, key = lambda item: (item['index'])):
            prices.append(i['data'])
        item = {
            'name': cate,
            'data': prices,
            'type': types 
        }
        yield item


In [24]:
categories = ['全新', '99成新', '95成新', '9成新', '8成新', '7成新及以下', '报废机/尸体']

options = dict(
    title=dict(text='成色价格统计'),
    chart=dict(zoomType='xy'),
    xAxis=dict(title=dict(text='成色'), categories=categories),
    yAxis=dict(title=dict(text='价格'))
)

series = [ i for i in generator('line')]

charts.plot(series, options=options, show='inline')